In [1]:
from keras.models import model_from_json
import tensorflow as tf
from keras import backend as K
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print(gpu)
else:
    print("No GPUs found.")
import librosa
import librosa.display
import numpy as np
import os

No GPUs found.


In [2]:

json_file = open('/Users/dikshantthapa/Desktop/GuitarTunes/GuitarTunes/models/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("/Users/dikshantthapa/Desktop/GuitarTunes/GuitarTunes/models/model.h5")
print("Loaded model from disk")

Loaded model from disk


In [3]:
def precision(y_true, y_pred):
   
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
   
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fbeta_score(y_true, y_pred, beta=1):
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')
        
    
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score


def fmeasure(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=1)

In [4]:
loaded_model.compile(
	optimizer="Adam",
	loss="categorical_crossentropy",
	metrics=['accuracy', precision, recall,fmeasure])

In [5]:
def record():
    import pyaudio
    import wave

    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    CHUNK = 1024
    RECORD_SECONDS = 5
    WAVE_OUTPUT_FILENAME = "/Users/dikshantthapa/Documents/FlutterDev/Sem_project/GuitarTunes/notebook/data/Music/Training/Am/Am_AcusticPlug15_4.wav"

    audio = pyaudio.PyAudio()

    # start Recording
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)
    print ("recording...")
    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    print ("finished recording")


    # stop Recording
    stream.stop_stream()
    stream.close()
    audio.terminate()

    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))
    waveFile.close()

In [7]:

y, sr = librosa.load('/Users/dikshantthapa/Documents/FlutterDev/Sem_project/server_tune/Am_AcousticGuitar_RodrigoMercador_1.wav',duration=2)
ps = librosa.feature.melspectrogram(y=y, sr=sr)
ps.shape
ps= np.array(ps.reshape(1,128,87,1))
ps.shape

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numba/np/ufunc/ufuncbuilder.py:328: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  self.nb_func = jit(_target='npyufunc', cache=cache)(py_func)


(1, 128, 87, 1)

In [8]:
predictions = loaded_model.predict(ps)
predictions

1/1 [==============================] - 0s 122ms/step


array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [9]:
# class_id=(predictions[0])
class_id = predictions[0]
class_id



array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [10]:
for i, output in enumerate(class_id):
    if output == 1.0:
        value = i
        break

value

1

In [12]:
classes = np.array(['a','am','bm','c','d','dm','e','em','f','g'])
classes[value]

'am'

In [13]:
#print(classes[class_id])
#if isinstance(class_id, int) and 0 <= class_id < len(classes):
#    print(classes[class_id])
#else:
#    print("Invalid class_id")

In [42]:

# from keras.models import model_from_json
# from keras import backend as K
# import librosa
# import librosa.display
# import numpy as np
# import os
# from tkinter import *
# from pygame import mixer
# mixer.init(44100)
# from PIL import Image, ImageTk
# import pandas as pd
# tk=Tk()
# import time

# canvas_width=500
# canvas_height=300

# canvas=Canvas(tk,width=canvas_width,height=canvas_height)
# canvas.pack() 

# import matplotlib.pyplot as plt
# import pylab


# def precision(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision


# def recall(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall


# def fbeta_score(y_true, y_pred, beta=1):
    
#     if beta < 0:
#         raise ValueError('The lowest choosable beta is zero (only precision).')
        
    
#     if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
#         return 0

#     p = precision(y_true, y_pred)
#     r = recall(y_true, y_pred)
#     bb = beta ** 2
#     fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
#     return fbeta_score


# def fmeasure(y_true, y_pred):

#     return fbeta_score(y_true, y_pred, beta=1)

# def restart_program():

#     python = sys.executable
#     os.execl(python, python, * sys.argv)

# def record():
#     import pyaudio
#     import wave

#     FORMAT = pyaudio.paInt16
#     CHANNELS = 2
#     RATE = 44100
#     CHUNK = 1024
#     RECORD_SECONDS = 3
#     WAVE_OUTPUT_FILENAME = "/Users/dikshantthapa/Documents/FlutterDev/Sem_project/GuitarTunes/notebook/data/recorded.wav"

#     audio = pyaudio.PyAudio()

    
#     stream = audio.open(format=FORMAT, channels=CHANNELS,
#                     rate=RATE, input=True,
#                     frames_per_buffer=CHUNK)
#     print ("recording...")
#     frames = []

#     for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
#         data = stream.read(CHUNK)
#         frames.append(data)
#     print ("finished recording")


#     # stop Recording
#     stream.stop_stream()
#     stream.close()
#     audio.terminate()

#     waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
#     waveFile.setnchannels(CHANNELS)
#     waveFile.setsampwidth(audio.get_sample_size(FORMAT))
#     waveFile.setframerate(RATE)
#     waveFile.writeframes(b''.join(frames))
#     waveFile.close()

# def classify():
#     # Example of a Siren spectrogram
#     y, sr = librosa.load('recorded.wav',duration=2)
#     ps = librosa.feature.melspectrogram(y=y, sr=sr,)
#     px=ps
#     ps.shape
#     ps= np.array(ps.reshape(1,128,87,1))
#     ps.shape
#     predictions = loaded_model.predict_classes(ps)
#     class_id=(predictions[0])
#     print("The recorded chord is "+str(classes[class_id]))
#     result=canvas.create_text(200,200,text="The recorded chord is "+str(classes[class_id]),font="Times 15 bold")




# def play():
#     mixer.Sound("/Users/dikshantthapa/Documents/FlutterDev/Sem_project/GuitarTunes/notebook/data/recorded.wav").play()
    
# # load json and create model
# json_file = open('/Users/dikshantthapa/Documents/FlutterDev/Sem_project/GuitarTunes/models/model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("/Users/dikshantthapa/Documents/FlutterDev/Sem_project/GuitarTunes/models/model.h5")
# print("Loaded model from disk")
# classes=['a','am','bm','c','d','dm','e','em','f','g']
        
# loaded_model.compile(
# 	optimizer="Adam",
# 	loss="categorical_crossentropy",
# 	metrics=['accuracy', precision, recall,fmeasure])
          

# ox,oy=0,0


# #Buttons
# record=Button(canvas, text='Record',font="Times 15 bold", command=record)
# canvas.create_window(150, 50, window=record, height=25, width=100)

# play=Button(canvas, text='Play',font="Times 15 bold", command=play)
# canvas.create_window(250, 50, window=play, height=25, width=50)

# classify=Button(canvas, text='Classify',font="Times 15 bold", command=classify)
# canvas.create_window(350, 50, window=classify, height=25, width=100)

# clear=Button(canvas, text='Clear',font="Times 15 bold", command=restart_program)
# canvas.create_window(250, 100, window=clear, height=25, width=100)



# mainloop()


pygame 2.4.0 (SDL 2.26.4, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html
Loaded model from disk


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/j0/kx0rsn7x28l84ccbn6ndhx5c0000gn/T/ipykernel_25268/3494620113.py", line 65, in record
    import pyaudio
ModuleNotFoundError: No module named 'pyaudio'
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/j0/kx0rsn7x28l84ccbn6ndhx5c0000gn/T/ipykernel_25268/3494620113.py", line 119, in play
    mixer.Sound("/Users/dikshantthapa/Documents/FlutterDev/Sem_project/GuitarTunes/notebook/data/recorded.wav").play()
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pygame.e